In [ ]:
%matplotlib notebook
import numpy as np 
import math
import random
import sys
from time import time
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.signal import savgol_filter
from scipy.stats import rv_continuous
from scipy.stats import gamma
from scipy.stats import moyal
from scipy.stats import poisson
from scipy.stats import binned_statistic_2d
from scipy.optimize import curve_fit
from scipy.interpolate import interp1d
from skimage.filters import gaussian
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
def total_rate(rate,loc):
    '''returns the rate of radio events per micro second'''
    fiducial_mass=1.16e7 #fiducial mass of Ar (in kg)
    CPA_mass=50 #kg
    APA_mass=258 #kg
    
    if loc=="active V":
        return fiducial_mass*rate/(1e6)
    elif loc=="CPA":
        return CPA_mass*rate/(1e6)
    elif loc=="APA":
        return APA_mass*rate/(1e6)

alpha=chr(945)
beta=chr(946)
gammachr=chr(947)
mu=chr(956)

Isotopes=["39-Ar",
          "42-Ar","42-K",
          "40-K","40-K",
          "60-Co","60-Co","60-Co",
          "85-Kr",
          "214-Pb","214-Bi","210-Pb","210-Bi"]
Decay_modes=[beta,
             beta,beta,
             beta,gammachr,
             beta,gammachr,gammachr,
             beta,
             beta,beta,beta,beta]
BqRates=[1.01,
         92e-6,92e-6,
         0.8928*4.9,0.1072*4.9,
         45.5e-3,45.5e-3,45.5e-3,
         0.115,
         40e-3,40e-3,40e-3,40e-3]
Energies=[0.565,
          0.59940,3.525,
          1.31,1.460,
          0.31,1.1732,1.3325,
          0.687,
          1.024,3.272,0.0635,1.1621]
Locations=["active V",
           "active V","active V",
           "CPA","CPA",
           "APA","APA","APA",
           "active V",
           "active V","active V","active V","active V"]

#create dataframe with data from radioactive isotopes
df_backgrounds=pd.DataFrame({'Isotope':Isotopes,
                'Decay mode':Decay_modes,
                'Location':Locations,
                'Rate (Bq/kg)':BqRates,
                'Energy (MeV)':Energies
                })

#calculate rates in the detector depending on location, and add it to dataframe
Total_rates=[]
for i in range (df_backgrounds.shape[0]):
    Total_rates.append(total_rate(df_backgrounds.iloc[i,3],df_backgrounds.iloc[i,2]))

df_backgrounds.insert(4, 'Rate ('+mu+'Bq)', Total_rates)
print(df_backgrounds)

In [ ]:
#lifetime=1000       #electron lifetime (micro-s)

class radiation_package():
    '''class to simulate the background decays from radioactive isotopes in DUNE'''
    
    def __init__(self,Q,isotope,lifetime):
        
        #parameters
        self.Q=Q                   #Q-value of the isotope (MeV)
        self.isotope=isotope       #name of the isotope
        self.ie = 23.6e-6          #ionisation energy of argon (MeV)
        self.v = 1.6               #electron velocity (mm/micro-s)
        #self.screen = screen      #position of event in the detector (mm)
        self.lifetime = lifetime   #electron lifetime (micro-s)
        self.max_dist=3594         #drift distance from CPA to APA in single phase DUNE module (mm)
        self.max_N = math.floor((self.Q/self.ie)*np.exp(-((1)/self.v)/self.lifetime)) #maximal electron count
        
                                
        #arrays used for computations and plots
        self.dist_array = np.linspace(1, self.max_dist, 1000) #discrete array of distance values
        self.energy_array = np.linspace(0, self.Q, 1000)      #discrete array of energy values
        
    
    
    def bds(self,Q):
        '''function to compute the beta decay spectrum of the isotope using Fermi Theory
        for possible energy values from 0 to the Q-value'''
        class bds_gen(rv_continuous):
            "create a continuous PDF of the beta decay spectrum"
            def _pdf(self, x):
                return (np.sqrt(x**2 + 2*x*0.511)*((Q-x)**2)*(x+0.511))
    
        return bds_gen(a=0,b=Q)

    
    def plot_bds(self):
        '''plot the beta decay spectrum'''
        plt.figure()
        plt.plot(self.energy_array, self.bds(self.Q).pdf(self.energy_array))
        plt.title("Beta Decay Spectrum of "+self.isotope)
        plt.ylabel("N($E_{k}$)")
        plt.xlabel("Electron $E_{k}$ (MeV)")
        #plt.savefig("Plots/"+"bds"+self.isotope+".png");
        

    def calc_count_prob_gamma(self):
      '''function to calculate the electron count values if the decay mode is gamma (only variable is distance)'''
        N=self.Q/self.ie
        distance=self.dist_array
        t=distance/self.v
        N*=np.exp(-t/self.lifetime)
        N=np.floor(N)
        #bin for each value of count to get weights for each count
        weights=np.histogram(N,bins=np.int(np.max(N)))[0]
        #normalise to get probabilities
        prob=weights/np.sum(weights)
        return N,prob

    def calc_count(self,energy,distance):
        '''calculates the values of electron count with discrete energy and distance arrays'''
        N=energy/self.ie             #divide by IE of argon
        t=distance/self.v            #convert to time
        N*=np.exp(-t/self.lifetime)  #apply lifetime reduction in charge
        return N
        

    def plot_count(self):
        '''plot the discrete values of count against energy and distance from APA'''
        #create meshgrid of points and calculate the discrete electron counts
        X, Y = np.meshgrid(self.energy_array, self.dist_array)
        N=self.calc_count(X,Y)
        
        #plot figure
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(X, Y, N)
        plt.title("Electron Count for "+self.isotope)
        ax.set_xlabel('Energy (MeV)')
        ax.set_ylabel('Distance (mm)')
        ax.set_zlabel('Electron Count')
        plt.show()
        fig.savefig("drive/My Drive/Plots/surface_count.png",bbox_inches = 'tight')
        #plt.savefig("Plots/"+"3D counts"+self.isotope+".png")

        
    def calc_prob(self,window_length):
        '''function to find and add the probabilities of getting a unique electron count'''
        #create an array that holds the weights of each discrete energy count
        weights=np.empty(0)
        pdf=self.bds(self.Q).pdf(self.energy_array)
        for i in range (len(self.energy_array)):
            weights=np.append(weights,pdf)
        
        #create meshgrid of points and calculate the discrete electron counts
        X, Y = np.meshgrid(self.energy_array, self.dist_array)
        N=self.calc_count(X,Y)
        #reshape and floor the electron count array
        N=np.reshape(N,len(self.energy_array)*len(self.dist_array))
        N=np.floor(N)
        
        #create array to hold the sum of weights for each possible count value
        sum_weights=np.zeros(self.max_N+1)
        #loop over all possible count values and find the indices with the same count value
        for i in range(self.max_N+1):
            indices=np.where(N == i)
            #loop over the indices to sum their weights
            for j in indices[0]:
                sum_weights[i]+=weights[j]
        #normalise the weights to get probability
        prob_nf=sum_weights/np.sum(sum_weights)
        
        #fit the discrete probabilities using savgol filter
        prob=savgol_filter(prob_nf, window_length, 2) #window_length is the number of points used for the fit,polynomial of order 2
        #set to 0 the negative probability values that result from the fit
        for i in range (len(prob)):
            if prob[i]<0:
                prob[i]=0
        #normalise the probabilities
        prob/=np.sum(prob)
        
        #array of all possible electron counts used for plotting
        counts=np.arange(0,self.max_N+1,1)
        
        return counts,prob_nf,prob        

    
    def plot_count_pdf(self,counts,prob_nf,prob):
        '''function to plot the probability of getting an electron count value'''
        plt.figure()
        plt.plot(counts, prob_nf,label="Discrete")
        plt.plot(counts,prob,label="Fit")
        plt.title("Probability of Electron Count for "+ self.isotope)
        plt.ylabel("Probability")
        plt.xlabel("Electron count")
        plt.legend()
        plt.tight_layout()
        plt.show()
        #plt.savefig("Plots/"+"PDF counts"+self.isotope+".png");

In [ ]:
def total_counts(lifetime):
  '''function to calculate the electron count distributions for each isotope and sum them up'''
  #store the values of count
  counts=[]
  #store the values of probability
  probabilities=[]
  #store the maximal count value
  maxcount=0

  #loop over all isotopes
  for index, row in df_backgrounds.iterrows():

      #calculte the distribution if the decay mode is beta decay
      if row[1]==beta:
          start=time()
          rp=radiation_package(row[5],row[0],lifetime)
          count, prob_nf, prob = rp.calc_prob(101)
          #rp.plot_count_pdf(count,prob_nf,prob)
          #weight each distribution by its rate in the detector
          prob*=row[4]
          #append counts and probabilities
          counts.append(count)
          probabilities.append(prob)
        
          end=time()
          print("Time for",row[0],": ",end-start)
        
      #calculte the electron counts values to be added (no distribution) if the decay mode is gamma decay
      elif row[1]==gammachr:
          start=time()
          rp=radiation_package(row[5],row[0],lifetime)
          count, prob =  rp.calc_count_prob_gamma()
          #weight each distribution by its rate in the detector
          prob*=row[4]
          #append counts and probabilities
          counts.append(count)
          probabilities.append(prob)
          end=time()
          print("Time for",row[0],": ",end-start)
     
      #store the maximal value of count
      if np.max(count)>maxcount:
          maxcount=np.max(count)

  #sum all normalised probabilities together to get the final distribution
  background_counts=np.arange(0,maxcount+1,1)
  background_probs=np.zeros(len(background_counts))
  #loop over all distributions
  for i in range(len(counts)):
    c=np.array(counts[i]) #values of count of this isotops
    p=np.array(probabilities[i]) #values of probability of this isotope
    #loop over all possible count values
    for j in background_counts:
        #find the indices where this distribution has a possible count value
        indices=np.where(c == j)
        #loop over indices where the count values matched and add the weights to the total distribution
        for k in indices[0]:
            background_probs[j]+=p[k]
            
  #normalise the total distribution
  background_probs/=np.sum(background_probs)

  #np.save("drive/My Drive/LArTPCdata/background_counts.npy",background_counts)
  #np.save("drive/My Drive/LArTPCdata/background_probs.npy",background_probs)

  return background_counts, background_probs

In [ ]:
#plot the total distribution with the best fit approximations from Gamma and Moyal

lifetime=1000

background_counts,background_probs=total_counts(lifetime)
#background_counts=np.load("drive/My Drive/LArTPCdata/background_counts.npy")
#background_probs=np.load("drive/My Drive/LArTPCdata/background_probs.npy")
rp_Ar_39=radiation_package(df_backgrounds.iloc[0,5],df_backgrounds.iloc[0,0],lifetime)
count_Ar_39, prob_nf_Ar_39, prob_Ar_39 = rp_Ar_39.calc_prob(101)
max_count_Ar=count_Ar_39[-1]

#find best fit parameter for Moyal Class
error=sys.float_info.max
scale_moyal=0
#loop over scale parameter values
for i in range(100,2000):
  #calculate Moyal at this scale and find its MSE
  Moyal=moyal(loc=np.argmax(background_probs),scale=i)
  err=mean_squared_error(background_probs[:max_count_Ar],Moyal.pdf(background_counts[:max_count_Ar]))
  #update the optimal scale if the MSE is minimised
  if err<error:
    error=err
    scale_moyal=i

print("Moyal scale parameter:",scale_moyal)
print("Moyal MSE:",error)

#find best parameter for gamma class
error=sys.float_info.max
scale_gamma=0
a_gamma=0
#loop over alpha and scale parameter values
for a in np.arange(1,1.5,0.01):
  for i in range(2000,3000):
    #calculate Gamma at this scale and find its MSE
    Gamma=gamma(a=a,loc=0,scale=i)
    err=mean_squared_error(background_probs[:max_count_Ar],Gamma.pdf(background_counts[:max_count_Ar]))
    #update the optimal alpha and scale if the MSE is minimised
    if err<error:
      error=err
      scale_gamma=i
      a_gamma=a

print("Gamma scale parameter:",scale_gamma)
print("Gamma alpha parameter:",a_gamma)
print("Gamma MSE",error)

#create classes with best fit parameters for plotting
Moyal=moyal(loc=np.argmax(background_probs),scale=scale_moyal)
moyal_probs=Moyal.pdf(background_counts)

Gamma=gamma(a=a_gamma,loc=0,scale=scale_gamma)
Gamma_prob=Gamma.pdf(background_counts)

plt.figure()
plt.plot(background_counts[:max_count_Ar],background_probs[:max_count_Ar],label="All")
plt.plot(count_Ar_39,prob_Ar_39,label=("$^{39}$"+"Ar"))
plt.plot(background_counts[:max_count_Ar],Gamma_prob[:max_count_Ar],label="Gamma")
plt.plot(background_counts[:max_count_Ar],moyal_probs[:max_count_Ar],label="Moyal")
plt.title("Probability of Electron Count for all Backgrounds")
plt.ylabel("Probability")
plt.xlabel("Electron Count")
plt.legend()
plt.tight_layout()
#plt.savefig("drive/My Drive/Plots/fittedcounts.png");

plt.figure()
plt.plot(background_counts,background_probs,label="All")
plt.plot(count_Ar_39,prob_Ar_39,label=("$^{39}$"+"Ar"))
plt.plot(background_counts,Gamma_prob,label="Gamma")
plt.plot(background_counts,moyal_probs,label="Moyal")
plt.title("Probability of Electron Count for all Backgrounds")
plt.ylabel("Probability")
plt.xlabel("Electron Count")
plt.legend()
plt.tight_layout()
#plt.savefig("drive/My Drive/Plots/fittedcountsall.png");

print("Memory allocation:",2*(background_probs.nbytes)/(1024**2),"MB")

In [ ]:
def find_best_params_gamma(lifetime):
  '''function to find the best fit gamma parameters for different values of lifetime, 
  in order to fit them to a curve and not to have to recalculate the distribution every time the lifetime is changed'''
  background_counts,background_probs=total_counts(lifetime)

  rp_Ar_39=radiation_package(df_backgrounds.iloc[0,5],df_backgrounds.iloc[0,0],lifetime)
  count_Ar_39, prob_nf_Ar_39, prob_Ar_39 = rp_Ar_39.calc_prob(101)
  max_count_Ar=count_Ar_39[-1]

  error=sys.float_info.max
  scale_gamma=0
  a_gamma=0

  #loop over alpha and scale parameter values
  for a in np.arange(0.5,2,0.02):
    for i in np.arange(10,5000,50):
      #calculate Gamma at this scale and find its MSE
      Gamma=gamma(a=a,loc=0,scale=i)
      err=mean_squared_error(background_probs[1:max_count_Ar],Gamma.pdf(background_counts[1:max_count_Ar]))
      #update the optimal gamma and scale if the MSE is minimised
      if err<error:
        error=err
        scale_gamma=i
        a_gamma=a

  Gamma=gamma(a=a_gamma,loc=0,scale=scale_gamma)
  Gamma_prob=Gamma.pdf(background_counts)
  plt.figure()
  plt.plot(background_counts[:max_count_Ar],background_probs[:max_count_Ar],label="All")
  plt.plot(background_counts[:max_count_Ar],Gamma_prob[:max_count_Ar],label="Gamma")

  return a_gamma,scale_gamma

In [ ]:
#calculate the best parameters of gamma for different lifetime values and fit them to a curve
a_array=[]
scale_array=[]
lifetimes=[500,1000,1500,2000,2500,3000]

for l in lifetimes:
  a,scale=find_best_params_gamma(l)
  a_array.append(a)
  scale_array.append(scale)

#use curve_fit to fit these values to a logarithmic curve
def func_log(x,m,n):
  return m*np.log(n*x)

popt_alpha, pcov_alpha = curve_fit(func_log, lifetimes, a_array)

popt_scale, pcov_scale = curve_fit(func_log, lifetimes, scale_array)


#plot the results
x=np.arange(100,3000,1)

color = 'tab:blue'
fig, ax1 =plt.subplots()
ax1.set_title("Parameters of Gamma Distribution against Electron Lifetime")
ax1.set_xlabel("Lifetime (ms)")
ax1.set_ylabel(chr(945),color=color)
ax1.plot(lifetimes,a_array,"o",color=color)
ax1.plot(x,func_log(x,*popt_alpha),color=color)
#ax1.plot(x,spline(x,lifetimes,a_array),color=color)
ax1.tick_params(axis='y', labelcolor=color)
#ax1.set_yscale('log')

ax2=ax1.twinx() # second axes with same x-axis

color = 'tab:orange'
ax2.set_ylabel("Scale",color=color)
ax2.plot(lifetimes,scale_array,"o",color=color)
ax2.plot(x,func_log(x,*popt_scale),color=color)
#ax2.plot(x,spline(x,lifetimes,scale_array),color=color)
ax2.tick_params(axis='y', labelcolor=color)
#ax2.set_yscale('log')

#plt.savefig("drive/My Drive/Plots/Gamma parameters.png",bbox_inches = 'tight')
plt.show();

In [ ]:
class SimpleLarTPCSim():
    """
    Main class to run the MARLEY neutrino event data simulation. Takes discrete arrays of possible electron counts and
    their associated probability for radioactive events as input.
    Returns TPC images with diffusion and electronic response effects
    """

    def __init__(self,screen,lifetime,rate_module,pdf):
        
        # load the neutrino event data
        self.electron_data    = np.load("drive/My Drive/forGiulio/marley.npy")
        #specify random seed for reproducibility
        np.random.seed(3)

        #parameters 
        self.ie = 23.6e-6          #ionisation energy of argon (MeV)
        self.v = 1.6               #electron velocity (mm/micro-s)
        self.screen = screen       #position of event in the detector (mm)
        self.lifetime = lifetime   #electron lifetime (micro-s)
        self.sigma_noise = 5       #standard deviation of gaussian used for thermal noise                     
        
        self.event_t=201                           #duration of simulated event (micro-s)
        self.event_len=240                         #half the length of cube of simulated event (mm)
        self.rate_module = rate_module             # rate of decay in a module per micro-s for all backgrounds
        self.event_vol = ((48/100)*self.event_len*2)**3     #volume of events
        self.vol_module = (1.3107e+13)/4           # volume of single phase module (mm^3)
        
        
        self.pdf=pdf   #gamma distribution of electron counts from all isotopes used to pick random background events
        self.rate =self.rate_module * self.event_vol/self.vol_module * ((48/100)*self.event_t) #rate of decay in the event volume
        self.poisson=poisson(self.rate) #Poisson PMF
        
        #Diffusion
        self.D_l=5.3e-4   #longitudinal diffusion coefficient (mm^2/micro-s)
        self.D_t=1.28e-3  #transverse diffusion coefficient (mm^2/micro-s)
        self.sigma_long=np.sqrt(2*self.D_l*(self.event_t/2)/(self.v**2)) #longitudinal diffusion standard deviation
        self.sigma_trans=2*np.sqrt(self.D_t*(self.event_t/2))            #transverse diffusion standard deviation
        
        #function to transform electron counts to ADC counts
        adc_range = [500,4096]
        hits_range = [0, 80000] #maximal hit is arbitrary, values above will  be max ADC (saturation)
        self.ADC = interp1d(hits_range, adc_range, bounds_error = False, fill_value = 4096)
        

        
    def getArrayOfImages(self,Nevents):
        curEv=0
        firstIndex=0
        numLines=np.shape(self.electron_data)[0]
        outputArray=np.zeros((Nevents,48,48))
        for i in range(numLines):
            if self.electron_data[i,4]!=curEv:
                outputArray[curEv]=self.processEventFast(curEv,firstIndex,i)
                curEv+=1
                firstIndex=i
                if(curEv>=Nevents):
                    break           
        return outputArray
              
    
    def processEventFast(self,event,first,last):

        event_data=self.electron_data[first:last,0:4] #fill x,y,z and edep array
        
        ymin=np.min(event_data[:,0])
        if ymin<0:
            event_data[:,0]-=ymin
        event_data[:,0]+=self.screen #difference between x-pos and screen
        event_data[:,0]/=self.v      #convert to time
        event_data[:,3]/=self.ie     #divide by IE of argon
        event_data[:,3]*=np.exp(-event_data[:,0]/self.lifetime) #apply lifetime reduction in charge

        # create TPC image
        #histogram with number of hits on each wire
        Yedge= np.arange(self.screen/self.v, self.screen/self.v+self.event_t,2)
        Xedge = np.arange(-self.event_len,self.event_len,4.79) #4.79 mm wire spacing
        signal = binned_statistic_2d(event_data[:,0], event_data[:,2], bins = [Yedge, Xedge], statistic = 'sum', values = event_data[:,3])[0]          
        
        #reduce the images to size 48x48 pixels rather than 100x100
        #find the maximal value and centre the image around it
        max=np.max(signal)
        if max==0:
          return np.zeros((48,48))
        #find the x-y position of the maximal value
        y_max,x_max=np.where(signal==max)
        #check the x and y position are not too close to the border
        if y_max<24:
          y_max=24
        elif y_max>75:
          y_max=75
        if x_max<24:
          x_max=24
        elif x_max>75:
          x_max=75
        #take a square around the maximal point
        signal=signal[np.int(y_max-24):np.int(y_max+24),np.int(x_max-24):np.int(x_max+24)]

        #RADIOLOGICAL BACKGROUND
        
        #sample number of events from Poisson PMF
        N_radio=self.poisson.rvs(size=1)[0]
        
        #sample random electron count taking probability into account
        radiodata=self.pdf.rvs(size=N_radio)
        #radiodata=np.random.choice(self.background_counts,p=self.background_probs,size=N_radio)

        # if np.array_equal(radiodata,np.empty(0))==False:
        #   print(radiodata)
        #   print(event)

        #randomly add background hits on signal
        for countr in radiodata:
            x=np.random.randint(0,48)
            y=np.random.randint(0,48)
            signal[x,y]+=countr
            
        #add diffusion effects
        signal=gaussian(signal,sigma=(self.sigma_long,self.sigma_trans),truncate=10,multichannel=False)
        
        
        #convert to ADC counts
        shape = signal.shape
        flat_sig = signal.flatten()
        signal = self.ADC(flat_sig)
        signal = signal.reshape((shape))
 

        #add gaussian smear in time due to electronic noise (standard deviation = 1 time bin)
        signal = gaussian(signal, sigma=(1,0), truncate = 10, multichannel=False)
        #add thermal electronic noise
        noise = np.random.normal(loc = 0.0, scale = self.sigma_noise, size = signal.shape)
        signal+=noise    
         
        return signal

def plot_image(image):
    plt.figure()
    plt.xlabel('Wire Number')
    plt.ylabel('Time (2 $\mu s$)')
    plt.imshow(image)
    cbar=plt.colorbar()
    cbar.set_label('ADC Count')
    #plt.savefig("drive/My Drive/Plots/image4.png")
    plt.show()

In [ ]:
#produce LArTPC images

lifetime=1000
start = time()
#generate the gamma PDF for electron counts with wanted lifetime value
Gamma=gamma(a=func_log(lifetime,*popt_alpha),loc=0,scale=func_log(lifetime,*popt_scale))
x = SimpleLarTPCSim(1797,1000,df_backgrounds.sum()[4],Gamma)
imageArray=x.getArrayOfImages(10000).astype('float32')

#remove images if there is no signal
indices=[]
for i in range(len(imageArray)):
  if np.max(imageArray[i])==0:
    indices.append(i)

imageArray=np.delete(imageArray,indices,axis=0)

end = time()
print(end-start)

print(len(imageArray))
plot_image(imageArray[78])
#np.save("drive/My Drive/LArTPCdata/SupernovaImages.npy",imageArray)
#np.save("drive/My Drive/LArTPCdata/indices.npy",indices)

In [ ]:
class BackgroundImages():
    """
    Main class to produce background images for the machine learning implementation. The same effects apply, however the neutrino events are not included
    """

    def __init__(self,screen,lifetime,rate_module,pdf):
        
        #specify random seed for reproducibility
        np.random.seed(0)

        #parameters 
        self.ie = 23.6e-6          #ionisation energy of argon (MeV)
        self.v = 1.6               #electron velocity (mm/micro-s)
        self.screen = screen       #position of event in the detector (mm)
        self.lifetime = lifetime   #electron lifetime (micro-s)
        self.sigma_noise = 5       #standard deviation of gaussian used for thermal noise                     
        
        self.event_t=97                            #duration of simulated event (micro-s)
        self.event_len=115                         #half the length of cube of simulated event (mm)
        self.rate_module = rate_module             # rate of decay in a module per micro-s for all backgrounds
        self.event_vol = (self.event_len*2)**3     #volume of events
        self.vol_module = (1.3107e+13)/4           # volume of single phase module (mm^3)
        
        
        #arrays of discrete counts and probabilities
        self.pdf=pdf #gamma distribution of electron counts from all isotopes used to pick random background events
        self.rate =self.rate_module * self.event_vol/self.vol_module * (self.event_t) #rate of decay in the event volume
        self.poisson=poisson(self.rate) #Poisson PMF
        
        #Diffusion
        self.D_l=5.3e-4   #longitudinal diffusion coefficient (mm^2/micro-s)
        self.D_t=1.28e-3  #transverse diffusion coefficient (mm^2/micro-s)
        self.sigma_long=np.sqrt(2*self.D_l*(self.event_t/2)/(self.v**2)) #longitudinal diffusion standard deviation
        self.sigma_trans=2*np.sqrt(self.D_t*(self.event_t/2))            #transverse diffusion standard deviation
        
        #function to transform electron counts to ADC counts
        adc_range = [500,4096]
        hits_range = [0, 80000] #maximal hit is arbitrary, values above will  be max ADC (saturation)
        self.ADC = interp1d(hits_range, adc_range, bounds_error = False, fill_value = 4096)
        
                   
        

        
    def getArrayOfImages(self,Nevents):
        outputArray=np.zeros((Nevents,48,48))
        count=0
        for i in range(Nevents):
            outputArray[count]=self.processEventFast()
            count+=1
        return outputArray
              
    
    def processEventFast(self):

        event_data=np.zeros((4,4))

        # create TPC image
        #histogram with number of hits on each wire
        Yedge= np.arange(self.screen/self.v, self.screen/self.v+self.event_t,2)
        Xedge = np.arange(-self.event_len,self.event_len,4.79) #4.79 mm wire spacing
        signal = binned_statistic_2d(event_data[:,0], event_data[:,2], bins = [Yedge, Xedge], statistic = 'sum', values = event_data[:,3])[0]            
        
               
        #RADIOLOGICAL BACKGROUND

        radiodata=np.empty(0)
        #while np.array_equal(radiodata,np.empty(0))==True:
            #sample number of events from Poisson PMF
        N_radio=self.poisson.rvs(size=1)[0]
        if N_radio==0:
          N_radio=1
            #sample random electron count taking probability into account
        radiodata=self.pdf.rvs(size=N_radio)
            #radiodata=np.random.choice(self.background_counts,p=self.background_probs,size=N_radio)
        
        #randomly add background hits on signal
        for countr in radiodata:
            x=np.random.randint(0,48)
            y=np.random.randint(0,48)
            signal[x,y]+=countr
      

        #add diffusion effects
        signal=gaussian(signal,sigma=(self.sigma_long,self.sigma_trans),truncate=10,multichannel=False)
        
        
        #convert to ADC counts
        shape = signal.shape
        flat_sig = signal.flatten()
        signal = self.ADC(flat_sig)
        signal = signal.reshape((shape))
 

        #add gaussian smear in time due to electronic noise (standard deviation = 1 time bin)
        signal = gaussian(signal, sigma=(1,0), truncate = 10, multichannel=False)
        #add thermal electronic noise
        noise = np.random.normal(loc = 0.0, scale = self.sigma_noise, size = signal.shape)
        signal+=noise    
         
        return signal    
        

def plot_image(image):
    plt.figure()
    plt.xlabel('Wire Number')
    plt.ylabel('Time (2 micro s)')
    plt.imshow(image)
    cbar=plt.colorbar()
    cbar.set_label('ADC')
    plt.show()
    #plt.savefig("Plots/Signal.png")

In [ ]:
#produce radioactive decay images

lifetime=1000
start = time()
#generate the gamma PDF for electron counts with wanted lifetime value
Gamma=gamma(a=func_log(lifetime,*popt_alpha),loc=0,scale=func_log(lifetime,*popt_scale))
y = BackgroundImages(1797,1000,df_backgrounds.sum()[4],Gamma)
BackgroundArray=y.getArrayOfImages(9662).astype('float32')
end = time()
print(end-start)

plot_image(BackgroundArray[0])
np.save("drive/My Drive/LArTPCdata/BackgroundImages.npy",BackgroundArray)

In [ ]:
def run_sim(screen,lifetime,rate):
  '''Function to run simulation of images for both supernova and radiological events
  for different values of distance from the APA (screen), lifetime and decay rate.
  Also saves the images to a selected file.'''
  for s in screen:
    for l in lifetime:
      for r in rate:
        start=time()
        Gamma=gamma(a=func_log(l,*popt_alpha),loc=0,scale=func_log(l,*popt_scale))
        x = SimpleLarTPCSim(s,l,r,Gamma)
        imageArray=x.getArrayOfImages(10000).astype('float32')

        indices=[]
        for i in range(len(imageArray)):
          if np.max(imageArray[i])==0:
            indices.append(i)
        imageArray=np.delete(imageArray,indices,axis=0)
        
        y = BackgroundImages(s,l,r,Gamma)
        BackgroundArray=y.getArrayOfImages(len(imageArray)).astype('float32')

        end=time()
        print(end-start)

        if r==df_backgrounds.sum()[4]:
          np.save("drive/My Drive/LArTPCdata/SupernovaImagesS"+f'{s}'+"L"+f'{l}'+".npy",imageArray)
          np.save("drive/My Drive/LArTPCdata/BackgroundImagesS"+f'{s}'+"L"+f'{l}'+".npy",BackgroundArray)
        else:
          np.save("drive/My Drive/LArTPCdata/SupernovaImagesS"+f'{s}'+"L"+f'{l}'+"R"+f'{r}'+".npy",imageArray)
          np.save("drive/My Drive/LArTPCdata/BackgroundImagesS"+f'{s}'+"L"+f'{l}'+"R"+f'{r}'+".npy",BackgroundArray)

In [ ]:
#create images for the lifetime and energy comparison

screen=[116,1797,3478]
lifetime=[100,200,300,500,800,1000,1500,2000,2500,3000]
rate=[df_backgrounds.sum()[4]]
run_sim(screen,lifetime,rate)

In [ ]:
#create images for the decay rate comparison

screen=[1797]
lifetime=[1000]
rate=[5,10,15,20,25,30,35,40,45,50,60,70,80,90,100,120,150,175,200,250,300,400,500,600,700,800,900,1000]
run_sim(screen,lifetime,rate)

In [ ]:
#create images for the background/supernova ratio comparison

lifetime=1000
start = time()
Gamma=gamma(a=func_log(lifetime,*popt_alpha),loc=0,scale=func_log(lifetime,*popt_scale))
y = BackgroundImages(1797,1000,df_backgrounds.sum()[4],Gamma)
BackgroundArray=y.getArrayOfImages(96620).astype('float32')
end = time()
print(end-start)

plot_image(BackgroundArray[0])
np.save("drive/My Drive/LArTPCdata/BackgroundImagesratio.npy",BackgroundArray)

In [ ]:
#create background images for the decay rate comparison with increased ratio

lifetime=1000
rate=[5,10,15,20,25,30,35,40,45,50,60,70,80,90,100,120,150,175,200,250,300,400,500,600,700,800,900,1000]
for r in rate:
  start = time()
  Gamma=gamma(a=func_log(lifetime,*popt_alpha),loc=0,scale=func_log(lifetime,*popt_scale))
  y = BackgroundImages(1797,1000,r,Gamma)
  BackgroundArray=y.getArrayOfImages(96620).astype('float32')
  end = time()
  print(end-start)

  #plot_image(BackgroundArray[0])
  np.save("drive/My Drive/LArTPCdata/BackgroundImagesratio"+f'{r}'+".npy",BackgroundArray)